# Model demo



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from preprocess import *
from spacy.lang.en import English
import os
import numpy as np
from utils import save_file, load_file, load_model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [2]:
MODEL_FLAG = 'bilstm_max'
state_file_path = f'weights/{MODEL_FLAG}/{MODEL_FLAG}_best.pth'

labels = ['neutral', 'entailment', 'contradiction']
vocab_file = 'NLI_vocab.pkl'
folder = 'saved_files'
if os.path.exists(f'{folder}/{vocab_file}'):
    print('loading vocab from file')
    vocab = load_file(vocab_file)
else:
    print('creating vocab with training set')
    train_split = preprocess(split='train')
    vocab = create_vocab(train_split)
    save_file(vocab, vocab_file)

vocab_size = len(vocab.mapping)
embeddings = align_vocab_with_glove(vocab)
model = load_model(embeddings, labels, vocab_size, device, MODEL_FLAG, state_file_path)

loading vocab from file
saved_files/NLI_vocab.pkl
loaded embeddings from file
loading bilstm_max


In [3]:
def transform_sentence(sent, vocab):
    nlp = English()
    tokenizer = nlp.tokenizer
    sent = list(tokenizer(sent.lower()))
    sent_ids = []
    for token in sent:
        sent_ids.append(vocab.mapping.get(token.text, 0))
    return sent_ids, [len(sent_ids)]

transform_sentence('This maps sentences to id', vocab)

([29982, 17706, 25860, 30238, 14643, 7790], [6])

In [25]:
def make_prediction(sent1, sent2, vocab, model, model_flag):
    print(f'premise: {sent1}')
    print(f'hypothesis: {sent2}')
    print("\n")
    sent_ids1, length1 = transform_sentence(sent1, vocab)
    sent_ids2, length2 = transform_sentence(sent2, vocab)
    sent1 = torch.tensor([sent_ids1])
    sent2 = torch.tensor([sent_ids2])
    if model_flag == 'bow':
        logits = model(sent_ids1, length1, sent_ids2, length2)
    elif model_flag == 'lstm' or model_flag == 'bilstm' or model_flag == 'bilstm_max':
        logits = model(sent1, length1, sent2, length2)

    probabilities = F.softmax(logits, dim=1)
    predicted_label = torch.argmax(probabilities, dim=1)
    labels = ['neutral', 'entailment', 'contradiction']
    # output tensor to readable probabilities
    prob_list = [round(prob, 3) for prob in probabilities.tolist()[0]]
    for label, prob in zip(labels, prob_list):
        print(f'predicted {label} with {prob} probability')
    print('\n')
    print(f'Therefore, predicted relation: {labels[predicted_label]}')
    return labels[predicted_label]

relation = make_prediction('bob loves you', 'he is outside', vocab, model, MODEL_FLAG)

premise: bob loves you
hypothesis: he is outside


predicted neutral with 0.739 probability
predicted entailment with 0.023 probability
predicted contradiction with 0.237 probability


Therefore, predicted relation: neutral
